In [7]:
import requests
from lxml import etree
from tqdm import tqdm
import json

In [2]:
r = requests.get('https://www.ke.com/city/')
html = etree.HTML(r.text)

city_list = []
cities = html.xpath('//ul[@class="city_list_ul"]//a')
for city in tqdm(cities):
    city_list.append({
        'city_name': city.xpath('text()')[0],
        'city_url': 'https:' + city.xpath('@href')[0] + '/ershoufang/',
        'city_host': 'https:' + city.xpath('@href')[0]
    })

100%|██████████| 592/592 [00:00<00:00, 12124.16it/s]


In [3]:
for idx, i in tqdm(enumerate(city_list)):
    r = requests.get(i['city_url'])

    html = etree.HTML(r.text)
    districts = html.xpath('//div[@data-role="ershoufang"]//a')
    
    city_districts = []
    for district in districts:
        city_districts.append({
            'district_name': district.xpath('text()')[0],
            'district_url': district.xpath('@href')[0]
        })
    
    city_list[idx]['districts'] = city_districts


592it [10:15,  1.09it/s]


In [4]:
for idx, i in tqdm(enumerate(city_list)):
    
    for d_idx, j in enumerate(i['districts']):
        r = requests.get(i['city_host'] + j['district_url'])
        
        
        html = etree.HTML(r.text)
        hotpots = html.xpath('(//div[@data-role="ershoufang"]/div)[2]/a')
        
        hotpot_container = []
        for hotpot in hotpots:
            item = {
                'hotpot': hotpot.xpath('text()')[0],
                'url': hotpot.xpath('@href')[0]
            }
            
            hotpot_container.append(item)
        
        city_list[idx]['districts'][d_idx]['hotpot'] = hotpot_container

592it [13:21,  1.35s/it]


In [10]:
with open('../spider/spider/city_list.json','w') as f: # 如果filename不存在会自动创建， 'w'表示写数据，写之前会清空文件中的原有数据！
    f.write(json.dumps({'cities': city_list}))